# Sintetic Data Factory for Route Optimization in a Delivery Company
# Author: [Juan Andrés Méndez Galvis](https://github.com/jabandersnatch)


> What is the Factory design pattern?

<i>The Factory Method Pattern defines an interface for creating an object, but lets subclasses decide which class to instantiate. Factory Method lets a class defer instantiation to subclasses. (Gang of Four)</i>

> How can we use the Factory design pattern to build sintetic data?

<i>We can use the Factory design pattern to create a set of classes that generate data for different scenarios. In this case, we will create a set of classes that generate data for a delivery company. We will create a class for each type of data that we want to generate, such as clients, products, routes, etc. Each class will have a method that generates the data. We will then create a factory class that will create instances of these classes and call their methods to generate the data. </i>


In [20]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor
import folium

class CVRPSyntheticDataFactory:
    def __init__(self, depots, city_boundary, vehicle_params, config, seed=None):
        """
        Initialize the synthetic data factory for CVRP.

        Parameters:
        - depots: List of tuples [(longitude, latitude), ...] representing depot locations.
        - city_boundary: GeoDataFrame or Shapely Polygon defining the operational city boundary.
        - vehicle_params: Dictionary defining vehicle types with their capacities, ranges, and counts.
        - config: Dictionary containing fine-tuning parameters, including:
            - 'client_location_mean': Mean distance from depot (meters).
            - 'client_location_std': Standard deviation of distance from depot (meters).
            - 'demand_distribution': Dictionary specifying the demand distribution and its parameters.
              Example:
              {
                  'type': 'normal',  # 'normal', 'poisson', 'uniform', etc.
                  'parameters': {
                      'mean': 5,
                      'std': 2,
                      'low': 1,    # Minimum demand value
                      'high': 10   # Maximum demand value
                  }
              }
            - 'client_vehicle_ratio': Number of clients per vehicle.
        - seed: Integer seed for random number generator to ensure reproducibility.
        """
        self.depots = depots
        self.city_boundary = city_boundary
        self.vehicle_params = vehicle_params
        self.config = config
        self.random_state = np.random.RandomState(seed)

        # Prepare depots and city boundary in projected coordinate system for distance calculations
        self._prepare_geometries()

    def _prepare_geometries(self):
        """
        Prepare depots and city boundary by projecting them into a metric coordinate system.
        """
        # Convert depots to GeoDataFrame
        depots_gdf = gpd.GeoDataFrame(
            {'geometry': [Point(lon, lat) for lon, lat in self.depots]},
            crs='EPSG:4326'
        )
        # Convert city boundary to GeoDataFrame if it's a Polygon
        if isinstance(self.city_boundary, Polygon):
            self.city_boundary = gpd.GeoDataFrame(
                {'geometry': [self.city_boundary]},
                crs='EPSG:4326'
            )
        elif isinstance(self.city_boundary, gpd.GeoSeries):
            self.city_boundary = gpd.GeoDataFrame(geometry=self.city_boundary)
            self.city_boundary.crs = 'EPSG:4326'

        # Project to UTM for distance calculations
        utm_crs = self.city_boundary.estimate_utm_crs()
        self.city_boundary = self.city_boundary.to_crs(utm_crs)
        self.depots_gdf = depots_gdf.to_crs(utm_crs)

        # Extract coordinates for use in client generation
        self.depots_coords = np.array(
            [(geom.x, geom.y) for geom in self.depots_gdf.geometry]
        )
        self.city_boundary_polygon = self.city_boundary.unary_union
        self.minx, self.miny, self.maxx, self.maxy = self.city_boundary.total_bounds

    def generate_clients(self, num_clients):
        """
        Generate client nodes with locations and demands within the city boundary.

        Parameters:
        - num_clients: Integer specifying the number of client nodes to generate.

        Returns:
        - clients_df: Pandas DataFrame containing client IDs, locations, and demands.
        """
        clients = []

        # Configuration parameters for client location distribution
        location_mean = self.config.get('client_location_mean', 5000)  # Default mean distance from depot
        location_std = self.config.get('client_location_std', 2000)    # Default standard deviation
        max_attempts = 1000  # To prevent infinite loops

        # Configuration for demand distribution
        demand_distribution = self.config.get('demand_distribution', {
            'type': 'uniform',
            'parameters': {
                'low': 1,
                'high': 10
            }
        })

        def create_client(client_id):
            attempts = 0
            while attempts < max_attempts:
                # Choose a depot randomly
                depot_idx = self.random_state.choice(len(self.depots_coords))
                depot_x, depot_y = self.depots_coords[depot_idx]

                # Generate distance from depot using normal distribution
                distance = self.random_state.normal(loc=location_mean, scale=location_std)
                # Ensure distance is positive
                distance = abs(distance)

                # Generate angle uniformly between 0 and 2*pi
                angle = self.random_state.uniform(0, 2 * np.pi)

                # Calculate client location
                x = depot_x + distance * np.cos(angle)
                y = depot_y + distance * np.sin(angle)

                point = Point(x, y)
                if self.city_boundary_polygon.contains(point):
                    # Assign demand based on the specified distribution
                    demand = self.generate_demand(demand_distribution)
                    return {
                        'ClientID': client_id,
                        'DepotID': depot_idx + 1,
                        'X': x,
                        'Y': y,
                        'Demand': demand
                    }
                attempts += 1
            # If max_attempts reached, raise an error
            raise ValueError(f"Could not generate a client within the city boundary after {max_attempts} attempts.")

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(create_client, cid) for cid in range(1, num_clients + 1)]
            for future in futures:
                clients.append(future.result())

        clients_df = pd.DataFrame(clients)

        # Convert coordinates back to longitude and latitude
        clients_gdf = gpd.GeoDataFrame(
            clients_df, geometry=gpd.points_from_xy(clients_df['X'], clients_df['Y']),
            crs=self.city_boundary.crs
        )
        clients_gdf = clients_gdf.to_crs('EPSG:4326')
        clients_df['Longitude'] = clients_gdf.geometry.x
        clients_df['Latitude'] = clients_gdf.geometry.y

        # Drop unnecessary columns
        clients_df.drop(columns=['X', 'Y'], inplace=True)

        return clients_df

    def generate_demand(self, distribution_config):
        """
        Generate a demand value based on the specified probability distribution.

        Parameters:
        - distribution_config: Dictionary specifying the distribution type and parameters.

        Returns:
        - demand: Integer demand value.
        """
        dist_type = distribution_config.get('type', 'uniform')
        params = distribution_config.get('parameters', {})

        # Get low and high values for demand limits
        low = params.get('low', 1)
        high = params.get('high', np.inf)  # Use infinity if 'high' not specified

        if dist_type == 'uniform':
            demand = self.random_state.randint(low, high + 1)
        elif dist_type == 'normal':
            mean = params.get('mean', 5)
            std = params.get('std', 2)
            demand = int(abs(self.random_state.normal(mean, std)))
        elif dist_type == 'poisson':
            lam = params.get('lam', 5)
            demand = self.random_state.poisson(lam)
        else:
            raise ValueError(f"Unsupported demand distribution type: {dist_type}")

        # Enforce demand limits
        demand = max(low, min(demand, high))

        return demand

    def generate_vehicles(self):
        """
        Generate vehicle information based on provided vehicle parameters.

        Returns:
        - vehicles_df: Pandas DataFrame containing vehicle types, capacities, ranges.
        """
        vehicles = []
        for vehicle_type, params in self.vehicle_params.items():
            mean_capacity = params['capacity']
            mean_range = params['range']
            count = params['count']

            # Standard deviations are 20% of the mean values
            std_capacity = mean_capacity * 0.2
            std_range = mean_range * 0.2

            for _ in range(count):
                capacity = self.random_state.normal(mean_capacity, std_capacity)
                capacity = max(1, capacity)  # Ensure capacity is at least 1

                range_ = self.random_state.normal(mean_range, std_range)
                range_ = max(1, range_)  # Ensure range is at least 1

                vehicles.append({
                    'VehicleType': vehicle_type,
                    'Capacity': capacity,
                    'Range': range_
                })

        vehicles_df = pd.DataFrame(vehicles)
        return vehicles_df

    def get_depots(self):
        """
        Get depot information.

        Returns:
        - depots_df: Pandas DataFrame containing depot IDs and locations.
        """
        depots_gdf = self.depots_gdf.to_crs('EPSG:4326')
        depots_data = [{
            'DepotID': idx + 1,
            'Longitude': geom.x,
            'Latitude': geom.y
        } for idx, geom in enumerate(depots_gdf.geometry)]

        depots_df = pd.DataFrame(depots_data)
        return depots_df

    def generate_dataset(self):
        """
        Generate the complete dataset including depots, clients, and vehicles.

        Returns:
        - dataset: Dictionary containing DataFrames for depots, clients, and vehicles.
        """
        depots_df = self.get_depots()
        vehicles_df = self.generate_vehicles()

        # Calculate the number of clients based on client_vehicle_ratio
        total_vehicles = len(vehicles_df)
        client_vehicle_ratio = self.config.get('client_vehicle_ratio', 5.0)
        num_clients = int(total_vehicles * client_vehicle_ratio)

        clients_df = self.generate_clients(num_clients)

        dataset = {
            'Depots': depots_df,
            'Clients': clients_df,
            'Vehicles': vehicles_df
        }
        return dataset

In [21]:

def plot_data(dataset, city_boundary):
    """
    Plot the depots, clients, and city boundary on an interactive map using Folium.
    """
    # Create a base map centered around the city boundary
    city_center = city_boundary.to_crs('EPSG:4326').geometry.centroid.iloc[0]
    m = folium.Map(location=[city_center.y, city_center.x], zoom_start=12)

    # Plot depots
    for _, row in dataset['Depots'].iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"Depot {row['DepotID']}",
            icon=folium.Icon(color='red', icon='home')
        ).add_to(m)

    # Plot clients
    for _, row in dataset['Clients'].iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=3,
            popup=f"Client {row['ClientID']} (Demand: {row['Demand']})",
            color='blue',
            fill=True,
            fill_opacity=0.6
        ).add_to(m)

    return m


In [22]:
import geopandas as gpd
from shapely.validation import explain_validity

def check_and_fix_geometries(city_boundary):
    """
    Check and fix invalid geometries in the GeoDataFrame.

    Parameters:
    - city_boundary: GeoDataFrame containing the city boundary or areas.

    Returns:
    - valid_city_boundary: GeoDataFrame with fixed geometries.
    """
    # Check for invalid geometries
    invalid_geometries = city_boundary[~city_boundary.is_valid]
    
    if len(invalid_geometries) > 0:
        print(f"Found {len(invalid_geometries)} invalid geometries.")
        
        # Explain the reason for invalid geometries
        for idx, row in invalid_geometries.iterrows():
            print(f"Invalid geometry at index {idx}: {explain_validity(row.geometry)}")

        # Attempt to fix invalid geometries using buffer(0) trick
        city_boundary['geometry'] = city_boundary['geometry'].buffer(0)

        # Re-check if any geometries are still invalid after fixing
        still_invalid = city_boundary[~city_boundary.is_valid]
        if len(still_invalid) > 0:
            print("Some geometries could not be fixed automatically.")
        else:
            print("All geometries have been fixed.")
    
    return city_boundary

# Example usage
city_boundary = gpd.read_file('GeoSpatialData/AUrb.shp')
city_boundary = city_boundary.to_crs('EPSG:4326')  # Ensure CRS is WGS84

# Check and fix invalid geometries
city_boundary = check_and_fix_geometries(city_boundary)


Found 14 invalid geometries.
Invalid geometry at index 533: Ring Self-intersection[-74.1193325159999 4.59845409100006]
Invalid geometry at index 2414: Ring Self-intersection[-74.0629682889999 4.67859052500006]
Invalid geometry at index 2778: Ring Self-intersection[-74.0496680799999 4.65612984000006]
Invalid geometry at index 2792: Self-intersection[-74.0553628562603 4.65151491798795]
Invalid geometry at index 3401: Ring Self-intersection[-74.0379721419999 4.68890885500008]
Invalid geometry at index 4230: Ring Self-intersection[-74.1026023559999 4.48488649500007]
Invalid geometry at index 4246: Too few points in geometry component[-74.1175640769999 4.48500309700006]
Invalid geometry at index 4333: Ring Self-intersection[-74.1380164029999 4.63223798500007]
Invalid geometry at index 4344: Too few points in geometry component[-74.1520703019999 4.63751884100009]
Invalid geometry at index 4394: Ring Self-intersection[-74.156907294 4.62122206700008]
Invalid geometry at index 4501: Ring Self-i

/home/jamendez/.pyenv/versions/mos/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received an unexpected geometry.  Either a polygon with interior rings, or a polygon with less than 4 points, or a non-Polygon geometry.  Return arguments as a collection.
  return ogr_read(
/home/jamendez/.pyenv/versions/mos/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Geometry of polygon of fid 4246 cannot be translated to Simple Geometry. All polygons will be contained in a multipolygon.
  return ogr_read(
/home/jamendez/.pyenv/versions/mos/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Geometry of polygon of fid 4344 cannot be translated to Simple Geometry. All polygons will be contained in a multipolygon.
  return ogr_read(


In [23]:
from shapely.geometry import Point
import numpy as np
import geopandas as gpd

def generate_depots(city_boundary, n_depots, seed=None):
    """
    Generate n_depots depots evenly spaced within the city boundary.

    Parameters:
    - city_boundary: GeoDataFrame with valid geometries.
    - n_depots: Number of depots to generate.
    - seed: Random seed for reproducibility.

    Returns:
    - depots: List of tuples [(longitude, latitude), ...] representing depot locations.
    """
    # Copy city_boundary to avoid modifying the original
    city_boundary = city_boundary.copy()

    # Fix invalid geometries if any
    city_boundary['geometry'] = city_boundary['geometry'].buffer(0)

    # Merge all polygons into a single polygon
    city_polygon = city_boundary.union_all()

    # Project to a metric CRS (e.g., UTM) for distance calculations
    utm_crs = city_boundary.estimate_utm_crs()
    city_polygon_utm = gpd.GeoSeries([city_polygon], crs=city_boundary.crs).to_crs(utm_crs).iloc[0]

    # Get the bounding box of the city polygon
    minx, miny, maxx, maxy = city_polygon_utm.bounds

    # Estimate grid spacing based on the desired number of depots
    total_area = city_polygon_utm.area
    num_points = n_depots * 10  # Generate more points to ensure enough within the polygon
    grid_spacing = np.sqrt(total_area / num_points)
    dx = dy = grid_spacing

    # Generate grid points within the bounding box
    x_points = np.arange(minx, maxx, dx)
    y_points = np.arange(miny, maxy, dy)
    grid_x, grid_y = np.meshgrid(x_points, y_points)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    # Convert grid points to shapely Point objects
    points = [Point(xy) for xy in grid_points]

    # Keep only points that are within the city polygon
    points_within_polygon = [point for point in points if city_polygon_utm.contains(point)]

    # Shuffle the points to ensure randomness
    random_state = np.random.RandomState(seed)
    random_state.shuffle(points_within_polygon)

    # Evenly select depots from the list of points
    total_points = len(points_within_polygon)
    if total_points < n_depots:
        raise ValueError("Not enough points within the polygon to select depots.")
    else:
        indices = np.linspace(0, total_points - 1, n_depots, dtype=int)
        selected_points = [points_within_polygon[i] for i in indices]

    # Convert the selected points back to latitude and longitude
    selected_points_gdf = gpd.GeoDataFrame(geometry=selected_points, crs=utm_crs)
    selected_points_gdf = selected_points_gdf.to_crs('EPSG:4326')

    depots = [(point.x, point.y) for point in selected_points_gdf.geometry]

    return depots




```py

n_depots = 12 

# Generate depots
depots = generate_depots(city_boundary, n_depots)

# Display the generated depots
print("Depots:")
for depot in depots:
    print(depot)
```

In [24]:

print("City Boundary:")
print(city_boundary.head())



City Boundary:
    AUrCodigo                          AUrAAdmini  AUrTPlano   AUrCPlano  \
0  160379B002  Memorando 2868 1963-07-04T00:00:00          1       446_4   
1  160202B003     Oficio 6242 1964-12-14T00:00:00          7     293_4-3   
2  160602B001     Oficio 4500 1967-06-20T00:00:00          1       175_4   
3  160204B001  Resolución 309 1988-07-22T00:00:00          7  474_4-11-1   
4  160340B003                     Dato No Hallado          7     235_4-7   

         AUrArea                                          AUrNombre  \
0   30865.154107                  URBANIZACIÓN BODEGAS DE ALMACENAR   
1   70366.003967                                     AUTOPISTA MUZU   
2  177596.034196     URBANIZACIÓN MUZU 1a. ETAPA-  OSPINA PEREZ SUR   
3    3098.089701  CONJUNTO RESIDENCIAL SAN REMO. MULTIFAMILIARES...   
4   95568.406722  URBANIZACIÓN VERAGUAS CENTRAL4° SECTOR - VERAG...   

   SHAPE_Leng    SHAPE_Area                                           geometry  
0    0.006801  2.513

```py

# Initialize factory with a seed for reproducibility
factory = CVRPSyntheticDataFactory(depots, city_boundary, vehicle_params, config, seed=42)

# Generate dataset
dataset = factory.generate_dataset()

# Access depots, clients, and vehicles data
depots_df = dataset['Depots']
clients_df = dataset['Clients']
vehicles_df = dataset['Vehicles']
```

In [25]:
import numpy as np
import pandas as pd
import geopandas as gpd

"""
Define vehicle parameters.
"""
vehicle_params = {
    'Gas Car': {'capacity': 120, 'range': 150, 'count': 10},
    'EV': {'capacity': 96, 'range': 1000, 'count': 7},
    'drone': {'capacity': 25, 'range': 15, 'count': 7}
}

# Define configuration parameters
config = {
    'client_location_mean': 5000,  # Mean distance from depot in meters
    'client_location_std': 2000,   # Standard deviation of distance
    'demand_distribution': {
        'type': 'poisson',
        'parameters': {
            'lam': 13,     # Lambda parameter for Poisson distribution
            'low': 5,     # Minimum demand value
            'high': 20    # Maximum demand value
        }
    },
    'client_vehicle_ratio': 2.0    # Number of clients per vehicle
}

# Get the depots from depots.csv
# DepotID,Longitude,Latitude
df_depots = pd.read_csv('multi_depots.csv')
df_clients = pd.read_csv('clients.csv')
df_vehicles = pd.read_csv('multi_vehicles.csv')
df_vehicles_data = pd.read_csv('vehicles_data.csv')

# Convert the df to a list of tuples
depots = [(row['Longitude'], row['Latitude']) for _, row in df_depots.iterrows()]


dataset = {
    'Depots': df_depots,
    'Clients': df_clients,
    'Vehicles': df_vehicles
}

# Plot the data
plot_data(dataset, city_boundary=city_boundary)

/tmp/ipykernel_20228/4100871394.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  city_center = city_boundary.to_crs('EPSG:4326').geometry.centroid.iloc[0]


In [26]:
# Save the dataset to CSV files
depots_df.to_csv('depots.csv', index=False)
clients_df.to_csv('clients.csv', index=False)
vehicles_df.to_csv('vehicles.csv', index=False)

In [32]:
import pandas as pd

# Data for Gas Cars
gas_cars = {
    "Vehicle": "Gas Car",
    "Freight Rate [COP/km]": 5000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 30000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 16000,
    "Recharge/Fuel Time [min/10 percent charge]": 0.1,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": 10,
}

# Data for Drones
drones = {
    "Vehicle": "Drone",
    "Freight Rate [COP/km]": 500,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 3000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 220.73,
    "Recharge/Fuel Time [min/10 percent charge]": 2,
    "Avg. Speed [km/h]": 40,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}

# Data for Solar EVs
solar_ev = {
    "Vehicle": "Solar EV",
    "Freight Rate [COP/km]": 4000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 21000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": None,
    "Recharge/Fuel Time [min/10 percent charge]": None,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}

# Combine into a dataframe
vehicles_data = pd.DataFrame([gas_cars, drones, solar_ev])

# display the dataframe
vehicles_data


,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/(gal or kWh)],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficency [kWh/km],Fuel Price [COP/gal]
0,Gas Car,5000,500,30000,16000.00,0.1,NaN,10.0,NaN,NaN
1,Drone,500,500,3000,220.73,2.0,40.0,NaN,0.15,NaN
2,Solar EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15,NaN


In [28]:
# Save the vehicles data to a CSV file
vehicles_data.to_csv('vehicles_data.csv', index=False)